# 5. 행정동 변수 생성

In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
data = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Seoul_location/종로구 실거래(13~18).xlsx")
benefit = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Factor/서울특별시_기초생활수급자.xlsx")
pop = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Factor/서울특별시_인구.xlsx")
parking = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Factor/서울특별시_주차공간.xlsx")
change = pd.read_excel("C:/Users/Yoo/Documents/Capstone/Factor/행정동명 변경.xlsx")

## 5-1 기초생활수급자 변수 생성

In [3]:
data['법정동'] = data['법정동'].astype(str).str.strip()

In [4]:
data['A_benefit'] = ''

In [5]:
# 행정동명 변경
for i,j in enumerate(benefit['동']):
    if j in change.data_out.values:
        x = np.where(change['data_out'].values==j)
        benefit.at[i,'동'] = change.at[x[0][0],'data_in']
    else:
        print('{}번째에서 문제발생'.format(i))

In [6]:
# 기초생활수급자 변수 생성
for i,(j,k) in enumerate(zip(data['년'],data['행정동'])):
    if j in benefit['기간'].values and k in benefit['동'].values:
        x = np.where((benefit['기간'].values == j) & (benefit['동'].values == k))
        data.at[i,'A_benefit'] = benefit.at[x[0][0],'총 수급자 인원']
    else:
        print('{}번째에서 문제발생'.format(i))

## 5-2 인구 변수 생성

In [7]:
data['A_pop_all'] = ''
data['A_pop_65'] = ''
data['A_pop_for'] = ''

In [8]:
# 행정동명 변경
for i,j in enumerate(pop['동']):
    if j in change.data_out.values:
        x = np.where(change['data_out'].values==j)
        pop.at[i,'동'] = change.at[x[0][0],'data_in']
    else:
        print('{}번째에서 문제발생'.format(i))

In [9]:
# 인구 변수 생성
for i,(j,k) in enumerate(zip(data['년'],data['행정동'])):
    if j in pop['기간'].values and k in pop['동'].values:
        x = np.where((pop['기간'].values == j) & (pop['동'].values == k))

        data.at[i,'A_pop_all'] = pop.at[x[0][0],'전체 계']
        data.at[i,'A_pop_65'] = pop.at[x[0][0],'65세이상고령자']
        data.at[i,'A_pop_for'] = pop.at[x[0][0],'등록 외국인 계']
    else:
        print('{}번째에서 문제발생'.format(i))

## 5-3 주차장 변수 생성

In [10]:
data['A_par_all'] = ''
data['A_par_apt'] = ''

In [11]:
# 행정동명 변경
for i,j in enumerate(parking['동']):
    if j in change.data_out.values:
        x = np.where(change['data_out'].values==j)
        parking.at[i,'동'] = change.at[x[0][0],'data_in']
    else:
        print('{}번째에서 문제발생'.format(i))

In [12]:
# 기간으로 정렬 및 인덱스 초기화
parking = parking.sort_values('기간', ascending =False).reset_index()

In [13]:
# 쉼표 제거
for i in range(len(parking)):
    parking.at[i,'면수 계'] = parking.at[i,'면수 계'].replace(',','')
    parking.at[i,'건축물부설 공동주택 면수'] = parking.at[i,'건축물부설 공동주택 면수'].replace(',','')

In [14]:
# 결측값 좌표 저장
ch1 = np.where((parking['면수 계'].values == '-') | (parking['면수 계'].values == '…'))
ch2 = np.where((parking['건축물부설 공동주택 면수'].values == '-') | (parking['건축물부설 공동주택 면수'].values == '…'))

In [15]:
# 면수 계열 결측값 채우기
for i in ch1[0]:
    year = parking.at[i,'기간']
    area = parking.at[i,'자치구']
    dist = parking.at[i,'동']
    for j in range(1,6):
        if parking.loc[(parking['기간']==(year+j)) & (parking['자치구']==area) & (parking['동']==dist),'면수 계'].values != '-' and parking.loc[(parking['기간']==(year+j)) & (parking['자치구']==area) & (parking['동']==dist),'면수 계'].values != '…':
            parking.at[i,'면수 계'] = int(int(parking.loc[(parking['기간']==(year+j)) & (parking['자치구']==area) & (parking['동']==dist), '면수 계'])*0.98)
            break

In [16]:
# 건축물부설 공동주택 면수열 결측값 채우기
for i in ch2[0]:
    year = parking.at[i,'기간']
    area = parking.at[i,'자치구']
    dist = parking.at[i,'동']
    for j in range(1,6):
        if parking.loc[(parking['기간']==(year+j)) & (parking['자치구']==area) & (parking['동']==dist),'건축물부설 공동주택 면수'].values != '-' and parking.loc[(parking['기간']==(year+j)) & (parking['자치구']==area) & (parking['동']==dist),'건축물부설 공동주택 면수'].values != '…':
            parking.at[i,'건축물부설 공동주택 면수'] = int(int(parking.loc[(parking['기간']==(year+j)) & (parking['자치구']==area) & (parking['동']==dist), '건축물부설 공동주택 면수'])*0.98)
            break
        else:
            continue

In [17]:
# 주차장 변수 생성
for i,(j,k) in enumerate(zip(data['년'],data['행정동'])):
    if j in parking['기간'].values and k in parking['동'].values:
        x = np.where((parking['기간'].values == j) & (parking['동'].values == k))

        data.at[i,'A_par_all'] = parking.at[x[0][0],'면수 계']
        data.at[i,'A_par_apt'] = parking.at[x[0][0],'건축물부설 공동주택 면수']
    else:
        print('{}번째에서 문제발생'.format(i))

In [18]:
data.to_excel('종로구 실거래(13~18).xlsx', index = False)